# Word2Vec

In [1]:
import gensim
gensim.__version__

'4.3.2'

In [2]:
!pip install nltk==3.8.1

## 영어 데이터 다운로드 및 전처리

In [3]:
import re
from lxml import etree  # xml 및 html 문서 처리 라이브러리
import urllib.request
import zipfile
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Word2Vec을 학습하기 위한 데이터 다운로드

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7ab0e6bdb850>)

In [6]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
# .xpath : 특정 요소 선택
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

현재 영어 텍스트가 content_text에 저장되어져 있습니다. 이에 대해서 NLTK의 sent_tokenize를 통해서 문장을 구분해봅시다.

In [7]:
len(content_text)

24062319

In [8]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [9]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

총 문장의 개수는 273,424개입니다.

In [10]:
print(f'총 샘플의 개수 : {len(result)}')

총 샘플의 개수 : 273424


In [11]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## 영어 Word2Vec 훈련시키기

In [12]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

여기서 Word2Vec의 하이퍼파라미터값

vector_size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.  
window = 컨텍스트 윈도우 크기  
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)  
workers = 학습을 위한 프로세스 수  
sg = 0은 CBOW, 1은 Skip-gram.  

In [13]:
model_result = model.wv.most_similar("man")
print(model_result)

# metric = 'euclidean'을 설정하면 euclid 거리 기반 유사 단어를 추출

[('woman', 0.8469401001930237), ('guy', 0.8153237104415894), ('lady', 0.7735413908958435), ('boy', 0.7611092329025269), ('girl', 0.7392730116844177), ('gentleman', 0.7374130487442017), ('kid', 0.7090216279029846), ('soldier', 0.7010562419891357), ('poet', 0.6813989281654358), ('rabbi', 0.6712331771850586)]


In [14]:
model.wv["man"]

array([-0.28701374, -2.0056164 , -0.7531477 , -0.9475187 ,  1.6379247 ,
       -0.25058505,  1.8321431 , -0.1267112 , -0.61342853,  0.8390355 ,
       -1.407613  , -1.3081931 ,  0.3708621 ,  0.14499614, -0.7970844 ,
       -0.01098662, -0.27118695, -0.55995035,  0.85449624, -0.3014113 ,
       -0.6459126 ,  1.5515872 ,  1.0958929 , -1.0339835 ,  1.3202654 ,
        0.20577517, -2.5836954 , -1.39903   , -0.56531715, -0.2501765 ,
       -0.02409626, -0.22847876,  0.49426463,  0.5373587 , -1.29098   ,
       -1.1155083 , -0.68831533, -0.80328774, -2.3261662 , -1.4961275 ,
        0.82538563, -0.8835235 ,  0.8823928 ,  1.6301711 ,  0.36954272,
       -0.2309883 , -0.62339586, -0.957187  , -1.6721259 , -0.17809974,
       -0.701449  , -1.3342993 , -0.08632158,  0.6234731 , -0.45522836,
        0.50343525,  0.35493362, -0.46921766, -1.1716487 , -1.0686364 ,
       -0.24541268,  0.3968998 ,  1.1201453 ,  0.4970842 , -1.9756619 ,
        0.21586989, -1.7858447 ,  0.39442852, -0.05470408,  1.46

In [15]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [16]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8469401001930237), ('guy', 0.8153237104415894), ('lady', 0.7735413908958435), ('boy', 0.7611092329025269), ('girl', 0.7392730116844177), ('gentleman', 0.7374130487442017), ('kid', 0.7090216279029846), ('soldier', 0.7010562419891357), ('poet', 0.6813989281654358), ('rabbi', 0.6712331771850586)]


In [17]:
# 현재 경로
%pwd

'/content'

## 한국어 데이터 다운로드 및 전처리

KoNLPy의 OKT 등은 형태소 분석 속도가 너무 느립니다. 그래서 Mecab을 설치하겠습니다.  
단, Mecab은 형태소 분석 속도는 빠르지만 설치하는데 시간이 좀 걸립니다.

In [18]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [19]:
import urllib.request
from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7ab0b0f2b490>)

In [21]:
train_data = pd.read_table('ratings.txt')

In [22]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [23]:
print(len(train_data)) # 리뷰 개수 출력

200000


In [24]:
# NULL 값 존재 유무
print(train_data.isnull().values.any())

True


In [25]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [26]:
print(len(train_data)) # 리뷰 개수 출력

199992


In [27]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    # 정규표현식에서 ^는 not을 의미한다

<ipython-input-27-255f9e07fc1d>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [28]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1


In [29]:
# 불용어 정의
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [30]:
# 형태소 분석기 mecab을 사용한 토큰화 작업 (다소 시간 소요)
mecab = Mecab()
tokenized_data = []
for sentence in train_data['document']:
    temp_X = mecab.morphs(sentence) # 토큰화 : 형태소 단위로 분석하고, 분석된 형태소들을 리스트 형태로 반
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [31]:
print(tokenized_data[:3])

[['어릴', '때', '보', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ'], ['디자인', '배우', '학생', '으로', '외국', '디자이너', '그', '일군', '전통', '통해', '발전', '해', '문화', '산업', '부러웠', '는데', '사실', '우리', '나라', '에서', '그', '어려운', '시절', '끝', '까지', '열정', '지킨', '노라노', '같', '전통', '있', '어', '저', '같', '사람', '꿈', '꾸', '이뤄나갈', '수', '있', '다는', '것', '감사', '합니다'], ['폴리스', '스토리', '시리즈', '부터', '뉴', '까지', '버릴', '께', '하나', '없', '음', '최고']]


## 한국어 Word2Vec 훈련시키기

[['나는', '사과를', 먹는다'], ['이', '영화', '는', '재밌어']]

In [32]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [33]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

(18134, 100)

In [34]:
print(model.wv.most_similar("최민식"))

[('안성기', 0.8710367679595947), ('박신양', 0.8524433970451355), ('김명민', 0.8502552509307861), ('신들린', 0.8501629829406738), ('송윤아', 0.8494343757629395), ('드니로', 0.8482893705368042), ('워싱턴', 0.846302330493927), ('한석규', 0.8417465090751648), ('설경구', 0.8392013907432556), ('송강호', 0.8391040563583374)]


In [35]:
model.wv['최민식']

array([ 0.06818914,  0.14573894, -0.13547668, -0.03756919, -0.1431819 ,
       -0.3447854 ,  0.20257823,  0.36686814, -0.09743028, -0.10119855,
        0.08697645, -0.39011297, -0.28047773,  0.03888668,  0.07685204,
       -0.12107087,  0.0199513 , -0.1573372 ,  0.24877965, -0.3050284 ,
        0.41054466, -0.01547786,  0.18363751,  0.28833503,  0.03087163,
        0.05110784,  0.06430748, -0.37092727, -0.05041329,  0.14025661,
        0.29702717,  0.02689285,  0.27637097, -0.08535016, -0.13081479,
        0.46991563,  0.1157412 ,  0.14980094, -0.20143457, -0.44782466,
       -0.00954036, -0.41412276,  0.3555956 ,  0.11808167,  0.32551643,
       -0.12025791, -0.02128032, -0.41727853,  0.2749931 , -0.08925699,
        0.07602874,  0.28004715,  0.09656721,  0.10993981, -0.11167667,
       -0.21432176, -0.08608571,  0.22103864,  0.0813588 , -0.03092557,
        0.20705676,  0.18965925, -0.43587917,  0.00655094, -0.26775062,
        0.05828132,  0.07504974,  0.14419976, -0.3144251 ,  0.32

In [36]:
print(model.wv.most_similar("히어로"))

[('슬래셔', 0.8676745295524597), ('호러', 0.8659170269966125), ('느와르', 0.8103594779968262), ('최고봉', 0.8048275709152222), ('무협', 0.7948607206344604), ('수사', 0.7939987182617188), ('블록버스터', 0.790453314781189), ('고어', 0.7900485992431641), ('하이틴', 0.7878480553627014), ('정통', 0.7868566513061523)]


In [37]:
# 영어 모델이 저장된 경로로 이동
%cd /content

/content


In [38]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('kor_w2v') # 모델 저장

### 영어 Word2Vec 시각화하기

In [39]:
!python -m gensim.scripts.word2vec2tensor --input eng_w2v --output eng-w2v

2024-03-04 02:06:04,208 - word2vec2tensor - INFO - running /usr/local/lib/python3.10/dist-packages/gensim/scripts/word2vec2tensor.py --input eng_w2v --output eng-w2v
2024-03-04 02:06:04,209 - keyedvectors - INFO - loading projection weights from eng_w2v
2024-03-04 02:06:05,593 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (21613, 100) matrix of type float32 from eng_w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2024-03-04T02:06:05.589230', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-6.1.58+-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}
2024-03-04 02:06:07,796 - word2vec2tensor - INFO - 2D tensor file saved to eng-w2v_tensor.tsv
2024-03-04 02:06:07,796 - word2vec2tensor - INFO - Tensor metadata file saved to eng-w2v_metadata.tsv
2024-03-04 02:06:07,798 - word2vec2tensor - INFO - finished running word2vec2tensor.py


시각화 사이트 : https://projector.tensorflow.org/

### 한국어 Word2Vec 시각화하기

In [40]:
!python -m gensim.scripts.word2vec2tensor --input kor_w2v --output kor_w2v

2024-03-04 02:06:08,931 - word2vec2tensor - INFO - running /usr/local/lib/python3.10/dist-packages/gensim/scripts/word2vec2tensor.py --input kor_w2v --output kor_w2v
2024-03-04 02:06:08,931 - keyedvectors - INFO - loading projection weights from kor_w2v
2024-03-04 02:06:09,979 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (18134, 100) matrix of type float32 from kor_w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2024-03-04T02:06:09.977863', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-6.1.58+-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}
2024-03-04 02:06:11,088 - word2vec2tensor - INFO - 2D tensor file saved to kor_w2v_tensor.tsv
2024-03-04 02:06:11,088 - word2vec2tensor - INFO - Tensor metadata file saved to kor_w2v_metadata.tsv
2024-03-04 02:06:11,090 - word2vec2tensor - INFO - finished running word2vec2tensor.py


# FastText

## Word2Vec의 OOV 문제 확인해보기

OOV 문제(Out-Of-Vocabulary Problem) : Vocabulary에 존재하지 않는 단어가 등장하는 문제

In [41]:
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # Word2Vec 모델 로드

In [42]:
model_result = loaded_model.most_similar("overacting")
print(model_result)

KeyError: "Key 'overacting' not present in vocabulary"

In [43]:
model_result = loaded_model.most_similar("memory")
print(model_result)

[('brain', 0.729761004447937), ('vision', 0.719856321811676), ('body', 0.7177096009254456), ('perception', 0.7010553479194641), ('cortex', 0.6838814616203308), ('imagination', 0.6741567254066467), ('skin', 0.6588777899742126), ('personality', 0.6585374474525452), ('consciousness', 0.6564449667930603), ('reputation', 0.6527174115180969)]


In [44]:
model_result = loaded_model.most_similar("memorry")
print(model_result)

KeyError: "Key 'memorry' not present in vocabulary"

In [45]:
model_result = loaded_model.most_similar("electrofishing")
print(model_result)

KeyError: "Key 'electrofishing' not present in vocabulary"

## FastText로 같은 단어에 대해서 테스트해보기
- oov 문제의 해결 여부 확인

In [46]:
from gensim.models import FastText
fasttext_model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [47]:
fasttext_model.wv.most_similar('overacting')

[('subtracting', 0.8995289206504822),
 ('distracting', 0.8869383335113525),
 ('contracting', 0.8796104788780212),
 ('impacting', 0.8684367537498474),
 ('extracting', 0.8632185459136963),
 ('interacting', 0.8608551621437073),
 ('overeating', 0.860124409198761),
 ('manipulating', 0.8584014177322388),
 ('dissecting', 0.8510473966598511),
 ('overarching', 0.8479328751564026)]

In [48]:
fasttext_model.wv.most_similar('memorry')

[('memo', 0.8229179382324219),
 ('memoir', 0.7640862464904785),
 ('nemo', 0.7575443387031555),
 ('forgery', 0.7537749409675598),
 ('rehearse', 0.7499282360076904),
 ('jimmy', 0.748651921749115),
 ('memory', 0.7462465763092041),
 ('nostalgic', 0.7426935434341431),
 ('tommy', 0.73346346616745),
 ('tattoo', 0.7323691844940186)]

In [49]:
fasttext_model.wv.most_similar("electrofishing")

[('electrolyte', 0.866932213306427),
 ('electrolux', 0.8635613918304443),
 ('electro', 0.8451380133628845),
 ('electroencephalogram', 0.8408480286598206),
 ('electroshock', 0.8382351994514465),
 ('electrogram', 0.830697238445282),
 ('electrochemical', 0.8273126482963562),
 ('electrons', 0.8174574971199036),
 ('electrode', 0.8169761300086975),
 ('electron', 0.8142736554145813)]

## 자모 단위 Fast Text

In [50]:
!pip install hgtk

In [51]:
# Facebook Research에서 제공하는 FastText
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

fatal: destination path 'fastText' already exists and is not an empty directory.
/content/fastText
make: Nothing to be done for 'opt'.
Processing /content/fastText
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4214525 sha256=3db7948a5b44075ffa4eab40e219d4b49162c8f7a2ead68ad92fc17457447206
  Stored in directory: /tmp/pip-ephem-wheel-cache-wt60dkv2/wheels/8b/05/af/3cfae069d904597d44b309c956601b611bdf8967bcbe968903
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


In [52]:
import hgtk

In [53]:
urllib.request.urlretrieve(r'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt', filename='ratings_total.txt')

('ratings_total.txt', <http.client.HTTPMessage at 0x7ab0c57be200>)

In [54]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print(f'전체 리뷰 갯수 : {len(total_data)}')

전체 리뷰 갯수 : 200000


In [55]:
total_data[:5]

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


hgtk의 checker를 통해 입력이 한글인지 아닌지 판단하여 True False를 반환

In [56]:
hgtk.checker.is_hangul('ㄱ')

True

In [57]:
hgtk.checker.is_hangul('5')

False

letter를 사용하여 음절을 자모 단위로 분리하거나, 자모의 시퀀스를 다시 음절로 조합

In [58]:
hgtk.letter.decompose('남')

('ㄴ', 'ㅏ', 'ㅁ')

In [59]:
hgtk.letter.compose('ㄴ', 'ㅏ')

'나'

한글이 아닌 입력이 들어오거나 음절로 조합할 수 없는 경우 NotHangulException 에러 발생

In [60]:
hgtk.letter.compose('ㄴ', 'ㄴ')

NotHangulException: No valid Hangul character index

### 데이터 전처리

hgtk.letter.decompose()를 사용하여 특정 단어가 들어오면 이를 초성, 중성, 종성으로 나누는 함수 word_to_jamo를 구현
- 종성이 없는 경우에는 종성이 없다는 것을 표시하기 위해 특수문자 '-' 삽입

In [61]:
def word_to_jamo(token):
    def to_special_token(jamo):
        if not jamo:
            return '-'
        else:
            return jamo

    decomposed_token = ''
    for char in token:
        try:
            # char(음절)을 초성, 중성, 종성으로 분리
            cho, jung, jong = hgtk.letter.decompose(char)

            # 자모가 빈 문자열일 경우 특수문자로 대
            cho = to_special_token(cho)
            jung = to_special_token(jung)
            jong = to_special_token(jong)
            decomposed_token = decomposed_token + cho + jung + jong

        # 만약 char(음절)이 한굴이 아닐 경우 자모를 나누지 않고 추가
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char

    return decomposed_token


In [62]:
word_to_jamo('남동생')

'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

In [63]:
word_to_jamo('여동생')

'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

In [64]:
# 형태소 분해
mecab = Mecab()

In [65]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [66]:
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [67]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [68]:
from tqdm import tqdm
tokenized_data = []

for sample in total_data['reviews'].to_list():
    tokenized_sample = tokenize_by_jamo(sample)
    tokenized_data.append(tokenized_sample)

In [69]:
tokenized_data[0]

['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']

jamo로 변환된 token을 다시 단어로 복원하는 함수

In [70]:
def jamo_to_word(jamo_sequence):
    tokenized_jamo = []
    index = 0

    # 1. 초기 입력
    while index < len(jamo_sequence):
        if not hgtk.checker.is_hangul(jamo_sequence[index]):
            tokenized_jamo.append(jamo_sequence[index])
            index += 1
        else:
            tokenized_jamo.append(jamo_sequence[index:index+3])
            index += 3

    # 2. 자모 단위 토큰화 완료
    word = ''
    try:
        for jamo in tokenized_jamo:
            # 초성, 중성, 종성의 묶음으로 추정되는 경우
            if len(jamo) == 3:
                if jamo[2] == '-':
                    # 종성이 존재하지 않는 경우
                    word += hgtk.letter.compose(jamo[0], jamo[1])
                else:
                    # 종성이 존재하는 경우
                    word += hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
            # 한글이 아닌 경우
            else:
                word += jamo
    # 복원(compose) 중 에러 발생 시 초기 입력 반환
    except Exception in exception:
        if type(exception).__name__ == 'NotHangulException':
            return jamo_sequence

    return word

In [71]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')

'남동생'

### FastText

자모 단위로 토큰화된 데이터를 가지고 FastText 학습/

In [72]:
import fasttext

FastText 학습을 위해 기존의 훈련 데이터를 txt 파일 형식으로 저장

In [73]:
with open('tokenized_data.txt', 'w') as out:
    for line in tqdm(tokenized_data, unit='line'):
        out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 283552.77line/s]


CBoW 방식으로 학습

In [74]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [75]:
# 자동으로 모델이 저장되지는 않는다.
model.save_model('fasttext.bin')

In [76]:
model = fasttext.load_model('fasttext.bin')

In [77]:
# 학습시 자모 단위로 분해하였기 때문에, 벡터 값을 확인하기 위해서 역시 자모 단위로 input 해
model[word_to_jamo('남동생')]

array([ 0.10053975,  0.5483576 ,  0.9365663 , -0.6376111 , -0.77265793,
       -0.4806171 ,  0.12574688,  0.77823585, -0.5412491 , -0.37715617,
        0.85062027,  0.27477103, -0.2019856 ,  0.34986034, -0.3151048 ,
        0.804251  , -0.18566827,  0.6056214 , -0.24055727,  0.37627244,
        0.6698768 , -0.5237914 ,  0.7521822 ,  0.02341442, -0.17149611,
       -0.5461813 ,  0.35431802,  1.0537186 ,  1.0209532 , -0.791429  ,
        0.68040353, -0.5201949 ,  0.2005935 , -0.5775247 ,  1.5114708 ,
       -0.2786645 , -0.3823303 , -0.23561004, -0.3335364 ,  0.12960154,
        0.38627267, -0.26394966,  0.18493988,  0.18794619, -0.5911801 ,
       -0.5651976 ,  0.17870744,  0.07417093,  0.6941467 , -0.35705242,
       -0.3284669 ,  0.09078441,  0.5500784 ,  0.19030026, -1.2988886 ,
        0.13273665,  0.27076134,  0.52283436, -0.65024686, -0.07252873,
       -0.06780522, -0.7910158 , -0.14931817,  0.7369413 ,  0.11122435,
        0.07620656, -0.14498606,  0.05974256, -0.25741562, -0.19

'남동생' 벡터와 가장 유사도가 높은 벡터 추출 (get_nearest_neighbors() 사용)

In [80]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.885690450668335, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.830272376537323, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7768639326095581, 'ㅊㅣㄴㄱㅜ-'),
 (0.7601721882820129, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7577266097068787, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7378281354904175, 'ㅈㅗ-ㅋㅏ-'),
 (0.7079845070838928, 'ㄴㅏㅁㅇㅏ-'),
 (0.6951063871383667, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.692351758480072, 'ㅅㅓㄴㅁㅜㄹ'),
 (0.6873787641525269, 'ㅇㅓㄴㄴㅣ-')]

In [81]:
def transform(word_sequence):
    return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [82]:
transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10))

[('동생', 0.885690450668335),
 ('남친', 0.830272376537323),
 ('친구', 0.7768639326095581),
 ('생일', 0.7601721882820129),
 ('남편', 0.7577266097068787),
 ('조카', 0.7378281354904175),
 ('남아', 0.7079845070838928),
 ('중학생', 0.6951063871383667),
 ('선물', 0.692351758480072),
 ('언니', 0.6873787641525269)]

In [83]:
transform(model.get_nearest_neighbors(word_to_jamo('남동쉥'), k=10))

[('남동생', 0.8874843120574951),
 ('남친', 0.8062516450881958),
 ('남매', 0.7669442892074585),
 ('남긴', 0.7446198463439941),
 ('남김', 0.7416554689407349),
 ('남녀', 0.7277371287345886),
 ('남짓', 0.7262501120567322),
 ('남기', 0.7070667147636414),
 ('남여', 0.6969592571258545),
 ('남겼', 0.6955801844596863)]

In [84]:
transform(model.get_nearest_neighbors(word_to_jamo('남동생ㅋ'), k=10))

[('남동생', 0.9321752190589905),
 ('남친', 0.7956069707870483),
 ('동생', 0.7827707529067993),
 ('친구', 0.7173915505409241),
 ('생일', 0.7079606056213379),
 ('조카', 0.696767270565033),
 ('남편', 0.6739811301231384),
 ('남아', 0.6728275418281555),
 ('언니', 0.6727986931800842),
 (',,?', 0.6597532033920288)]

In [85]:
transform(model.get_nearest_neighbors(word_to_jamo('남동ㅅ'), k=10))

[('남동생', 0.7476889491081238),
 ('남친', 0.6642265915870667),
 ('남김', 0.6554815173149109),
 ('남짓', 0.6172954440116882),
 ('남녀', 0.6166591644287109),
 ('남긴', 0.6012113094329834),
 ('남길', 0.5916052460670471),
 ('남겼', 0.5874341726303101),
 ('남매', 0.5830644369125366),
 ('남음', 0.5743190050125122)]